In [6]:
import pandas as pd
import numpy as np
from datetime import datetime

In [7]:
tasks_table = pd.read_csv('additional tables/tasks_table.csv')
hero_marks_mean = pd.read_csv('additional tables/hero_marks_mean.csv')
hero_abilities = pd.read_csv('additional tables/hero_abilities.csv')
average_duration = pd.read_csv('additional tables/average_duration.csv')

tasks_table['Дата поручения']=tasks_table['Дата поручения'].str.split(' ').str[0].astype(str)
def safe_parse_date(date_str):
    try:
        return datetime.strptime(date_str, "%Y-%m-%d")
    except ValueError:
        return None 
    
tasks_table['Дата поручения']=tasks_table['Дата поручения'].apply(safe_parse_date)

## Рекомендательная система

Наша система должна предлагать лучшего кандидата(ов) для задачи в зависимости от покрытия целевых навыков, средних оценок по задаче и скорости выполнения.

Пример: для группы поручений 3 (очень опасные) нужно иметь навык лекаря, рекомендательная система не должна предлагать героев, у которых этот навык равен 0. Либо она может предложить команду, где обязательно один герой должен иметь навык лекаря

Мы определились, что нашим критерием для набора команды будет покрытие всех целевых действий. Если один герой покрывает все действия своими навыками, то ему не нужна команда, он может справиться сам. Плюсом это экономит человекоресурсы и оптимизирует выполнение задач, так как остаётся больше незанятых персонажей.

Рексис предлагает список лучших по посчитанным метрикам герое, а также комбинации героев в зависимости от группы задач. 

#### Показательный пример.

In [8]:

# Функция для подбора кандидатов по задаче
import pandas as pd
def recommend_candidates(task_group, hero_marks_mean, hero_abilities):
    # Определение целевых действий по группе задач
    group_actions = {
        0: ['разжечь костёр', 'залечить раны', 'выследить цель'],
        1: ['найти пропажу', 'отыскать заказчика'],
        2: ['выследить цель'],
        3: ['разжечь костёр', 'залечить раны'],
        4: ['разжечь костёр', 'выследить цель']
    }
    
    # Получение целевых действий для данной группы
    target_actions = group_actions.get(task_group, [])

    # Фильтрация героев, которые обладают нужными навыками
    relevant_heroes = hero_abilities[['Герой'] + target_actions].copy()

    # Подсчет средней оценки по целевым действиям для всех героев
    avg_action_scores = relevant_heroes[target_actions].mean()

    # Подсчет средней оценки по целевым действиям для каждого героя
    relevant_heroes['Skill Score'] = relevant_heroes[target_actions].mean(axis=1)

    # Группировка данных `hero_data` для получения уникальных записей по каждому герою
    unique_hero_data = hero_marks_mean.groupby('Герой').agg({
        'Оценка за качество': 'mean',
        'Оценка по срокам': 'mean',
        'Оценка за вежливость': 'mean',
        'Затрачено дней': 'mean'
    }).reset_index()

    # Объединение с данными о героях по оценкам качества, срокам, вежливости и времени выполнения
    candidate_data = pd.merge(relevant_heroes, unique_hero_data, on='Герой')

    # Нормализация оценки по срокам (чем меньше значение, тем лучше)
    candidate_data['Normalized Time'] = 1 / (candidate_data['Затрачено дней'] + 1e-5)

    # Итоговый рейтинг, объединяющий все оценки (веса можно подкорректировать)
    candidate_data['Final Score'] = (
        candidate_data['Skill Score'] * 0.4 +  # Вес на навыки
        candidate_data['Оценка за качество'] * 0.2 +  # Вес на качество
        candidate_data['Оценка по срокам'] * 0.25 +  # Вес на оценку срок выполнения
        candidate_data['Оценка за вежливость'] * 0.05 +  # Вес на вежливость
        candidate_data['Normalized Time'] * 0.1  # Вес на время выполнения
    )

    # Сортировка по итоговому рейтингу в порядке убывания
    sorted_candidates = candidate_data.sort_values(by='Final Score', ascending=False)

    # Список для хранения окончательных рекомендаций
    recommended_candidates = sorted_candidates[['Герой', 'Final Score']].copy()

    # Добавление комбинаций героев для покрытия всех действий
    combinations = []
    added_combinations = set()  # Множество для хранения уникальных комбинаций героев
    used_heroes = set()  # Множество для героев, уже включенных в комбинации
    
    for _, hero in sorted_candidates.iterrows():
        # Проверка, покрывает ли герой все целевые действия
        covers_all_actions = all(hero[action] >= avg_action_scores[action] for action in target_actions)
        
        if covers_all_actions and hero['Герой'] not in used_heroes:
            # Если герой покрывает все действия, добавляем его как одиночного кандидата
            combinations.append({
                'Герой': hero['Герой'],
                'Final Score': hero['Final Score']
            })
            used_heroes.add(hero['Герой'])  # Добавляем героя в список использованных
        else:
            # Если герой не покрывает все действия, ищем дополнительного героя для покрытия недостающих
            for _, additional_hero in sorted_candidates.iterrows():
                if additional_hero['Герой'] != hero['Герой'] and additional_hero['Герой'] not in used_heroes:
                    # Создаем комбинацию имен героев в алфавитном порядке для уникальности
                    hero_pair = tuple(sorted([hero['Герой'], additional_hero['Герой']]))
                    
                    # Проверка на дублирование комбинации
                    if hero_pair not in added_combinations:
                        # Проверка, может ли второй герой дополнить недостающие навыки
                        covers_missing_actions = all(
                            (hero[action] >= avg_action_scores[action] or additional_hero[action] > 0)
                            for action in target_actions
                        )
                        
                        if covers_missing_actions:
                            # Если пара героев покрывает все действия, рассчитываем общий скор комбинации
                            combined_score = (hero['Final Score'] + additional_hero['Final Score']) / 2
                            combinations.append({
                                'Герой': f"{hero['Герой']} & {additional_hero['Герой']}",
                                'Final Score': combined_score
                            })
                            added_combinations.add(hero_pair)  # Добавляем комбинацию, чтобы не повторять
                            used_heroes.update([hero['Герой'], additional_hero['Герой']])  # Добавляем героев в использованные
                            break

    # Добавляем комбинации к индивидуальным рекомендациям
    combined_df = pd.DataFrame(combinations)
    full_recommendations = pd.concat([recommended_candidates, combined_df], ignore_index=True)

    # Сортируем итоговую таблицу по скору
    full_recommendations = full_recommendations.sort_values(by='Final Score', ascending=False).reset_index(drop=True)
    full_recommendations = full_recommendations.drop_duplicates(subset=['Герой'], keep='first')
    return full_recommendations

# Пример использования функции для задачи из группы. Можно тыкать разные от 0 до 4 чтобы посмотреть работоспособность
task_group = 0
recommended_candidates = recommend_candidates(task_group, hero_marks_mean, hero_abilities)
print(recommended_candidates)


                 Герой  Final Score
0             Бенедикт     3.346360
2                Агата     2.966248
3         Агата & Соня     2.909233
4                 Соня     2.852217
5    Соня & Синеглазый     2.839361
6           Синеглазый     2.826504
7    Синеглазый & Юлия     2.806495
8                 Юлия     2.786486
9       Юлия & Альфред     2.781998
10             Альфред     2.777509
11    Альфред & Мартин     2.681228
12             Глюкоза     2.677850
13              Мартин     2.584946
14              Бендер     2.514006
15            Пастушок     2.496680
16  Глюкоза & Фредерик     2.473917
17            Леопольд     2.437742
18            Фредерик     2.269983


#### Рабочий пример рексис

In [10]:
import pandas as pd
from models.task_assignment import assign_tasks
from models.recommendation import recommend_candidates

# Необходимая инициализация
data = {
    'Герой': ['Агата', 'Альфред', 'Бендер', 'Бенедикт', 'Глюкоза', 'Леопольд', 'Мартин', 'Пастушок', 'Синеглазый', 'Соня', 'Фредерик', 'Юлия'],
    'Доступен': [None, None, None, None, None, None, None, None, None, None, None, None]
}
availability = pd.DataFrame(data) 
availability['Доступен'] = None
tasks_table['Предполагаемая дата выполнения']=None

# Если хотите чтобы герои отдыхали подольше после каждого задания, то укажите rest_days побольше
tasks_table = assign_tasks(tasks_table, availability, average_duration, hero_marks_mean, hero_abilities, rest_days=2)
tasks_table

,Unnamed: 0,Номер поручения,Заказчик,Дата поручения,Предполагаемая дата выполнения,Затрачено дней,Сумма вознаграждения,Исполнитель,group
0,0,11134,Иван,1053-09-04 00:00:00,1053-09-09 00:00:00,5.0,27500,Бенедикт,0
1,1,11056,Мария,1053-09-06 00:00:00,1053-09-12 00:00:00,6.0,23500,Агата,1
2,2,11381,Егор,1053-09-20 00:00:00,1053-09-25 00:00:00,5.0,7000,Бенедикт,0
3,3,11311,Бабушка Синь,1053-09-20 00:00:00,1053-09-26 00:00:00,6.0,19000,Агата,1
4,4,11417,Леонтия,1053-09-22 00:00:00,1053-09-29 00:00:00,7.0,20500,Альфред,1
5,5,11310,Надя,1053-09-24 00:00:00,1053-09-29 00:00:00,5.0,5000,Мартин,2
6,6,11285,Чарли,1053-09-30 00:00:00,1053-10-06 00:00:00,6.0,5000,Агата,1
7,7,11218,Олег,1053-10-01 00:00:00,1053-10-04 00:00:00,3.0,16000,Соня,2
8,8,11161,Эмилио,1053-10-03 00:00:00,1053-10-10 00:00:00,7.0,7500,Альфред,1
9,9,11387,Татьяна,1053-10-13 00:00:00,1053-10-19 00:00:00,6.0,6500,Агата,1
